<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/Lucia/TrainAndTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [0]:
import os
import logging
import sys


In [2]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone -b Lucia https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 495 (delta 102), reused 0 (delta 0), pack-reused 330
Receiving objects: 100% (495/495), 324.11 KiB | 607.00 KiB/s, done.
Resolving deltas: 100% (301/301), done.


# Import packages

In [0]:
from DatasetCIFAR.data_set import Dataset 
from DatasetCIFAR import ResNet
from DatasetCIFAR import utils
from DatasetCIFAR import params
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np

from torchvision import transforms
from torch.utils.data import Subset, DataLoader
from torch.nn import functional as F
import random
random.seed(params.SEED)

# Define Network

In [0]:
resNet = ResNet.resnet32(num_classes=100)
resNet = resNet.to(params.DEVICE)

In [0]:
resnet_transformer = transforms.Compose([transforms.Resize(32), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])


# Define DataSets

In [6]:
trainDS = Dataset(train=True, transform = resnet_transformer)
testDS = Dataset(train=False, transform = resnet_transformer)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [7]:
train_splits = trainDS.splits
test_splits = testDS.splits
print(train_splits)
print(test_splits)

[[49.0, 97.0, 53.0, 5.0, 33.0, 65.0, 62.0, 51.0, 38.0, 61.0], [55.0, 59.0, 6.0, 35.0, 74.0, 71.0, 57.0, 41.0, 70.0, 48.0], [64.0, 69.0, 7.0, 37.0, 84.0, 81.0, 67.0, 44.0, 80.0, 56.0], [75.0, 79.0, 8.0, 40.0, 94.0, 91.0, 77.0, 87.0, 76.0, 98.0], [90.0, 28.0, 82.0, 93.0, 30.0, 2.0, 20.0, 42.0, 39.0, 29.0], [34.0, 96.0, 43.0, 3.0, 22.0, 54.0, 52.0, 36.0, 25.0, 50.0], [58.0, 63.0, 4.0, 24.0, 83.0, 78.0, 60.0, 31.0, 73.0, 17.0], [92.0, 21.0, 86.0, 23.0, 1.0, 15.0, 32.0, 27.0, 95.0, 16.0], [46.0, 47.0, 66.0, 12.0, 19.0, 10.0, 26.0, 72.0, 11.0, 9.0], [18.0, 89.0, 0.0, 68.0, 14.0, 13.0, 45.0, 99.0, 88.0, 85.0]]
[[49.0, 97.0, 53.0, 5.0, 33.0, 65.0, 62.0, 51.0, 38.0, 61.0], [55.0, 59.0, 6.0, 35.0, 74.0, 71.0, 57.0, 41.0, 70.0, 48.0], [64.0, 69.0, 7.0, 37.0, 84.0, 81.0, 67.0, 44.0, 80.0, 56.0], [75.0, 79.0, 8.0, 40.0, 94.0, 91.0, 77.0, 87.0, 76.0, 98.0], [90.0, 28.0, 82.0, 93.0, 30.0, 2.0, 20.0, 42.0, 39.0, 29.0], [34.0, 96.0, 43.0, 3.0, 22.0, 54.0, 52.0, 36.0, 25.0, 50.0], [58.0, 63.0, 4.0, 24.0

# Useful plots

The function plotEpoch plots, at the end of each task, how accuracy and loss change during the training phase. It show

*   Validation and Training Accuracy
*   Validation and Training Loss

The function plotTask, for each task, how the accuracy on the validation set change when adding new tasks


In [0]:
def plotTask(pars_tasks):
  import numpy as np
  import matplotlib.pyplot as plt

  x_tasks =  np.linspace(10, 100, 10)

  plt.plot(x_tasks, pars_tasks, label=['Accuracy', 'Loss'])
  plt.xlabel("Epoch")
  plt.title('Accuracy over tasks')
  plt.legend(['Accuracy', 'Loss'])
  plt.grid(True)
  plt.show()



# Train and evaluation phase

In [0]:
pars_tasks = []
test_indexes = []

for task in range(0, 100, params.TASK_SIZE):
  pars_tasks.insert(task, 0)

for task in range(0, 100, params.TASK_SIZE):

  train_indexes = trainDS.__getIndexesGroups__(task)
  test_indexes = test_indexes + testDS.__getIndexesGroups__(task)

  train_dataset = Subset(trainDS, train_indexes)
  test_dataset = Subset(testDS, test_indexes)

  train_loader = DataLoader( train_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE)
  test_loader = DataLoader( test_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE )

  if(task == 0):
    torch.save(resNet, 'resNet_task{0}.pt'.format(task))
  
  

  utils.trainfunction(task, train_loader, train_splits)
  param = utils.evaluationTest(task, test_loader, test_splits) #evaluate test set at step task
  pars_tasks[int(task/10)] = param #pars_task[i] = (accuracy, loss) at i-th task	

train_splits = trainDS.splits
test_splits = testDS.splits

task = 0 
train col =  [49 97 53  5 33 65 62 51 38 61]
train col =  [[49 97 53  5 33 65 62 51 38 61]]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


At step  0  and at epoch =  0  the loss is =  0.03358946368098259  and accuracy is =  0.1908
At step  0  and at epoch =  1  the loss is =  0.023538200184702873  and accuracy is =  0.3548
At step  0  and at epoch =  2  the loss is =  0.019897596910595894  and accuracy is =  0.4288
At step  0  and at epoch =  3  the loss is =  0.01805814355611801  and accuracy is =  0.4576
At step  0  and at epoch =  4  the loss is =  0.016763538122177124  and accuracy is =  0.4838
At step  0  and at epoch =  5  the loss is =  0.015761978924274445  and accuracy is =  0.504
At step  0  and at epoch =  6  the loss is =  0.013972810469567776  and accuracy is =  0.5188
At step  0  and at epoch =  7  the loss is =  0.01271007303148508  and accuracy is =  0.5418
At step  0  and at epoch =  8  the loss is =  0.01113803405314684  and accuracy is =  0.5592
At step  0  and at epoch =  9  the loss is =  0.01016147993505001  and accuracy is =  0.5752
At step  0  and at epoch =  10  the loss is =  0.00924897380173206

In [0]:
train_loader.shape